In [1]:
%reload_ext autoreload
%autoreload 2

from dataset.graph_builder import GraphBuilder

builder = GraphBuilder(dataset_path="data/MOLWENI/test.json", dataset_name="MOLWENI")
builder()

2025-02-07 14:57:02,193 - INFO - Loading dataset from data/MOLWENI/test.json
2025-02-07 14:57:02,199 - INFO - MOLWENI dataset loaded successfully.
2025-02-07 14:57:02,221 - INFO - Use pytorch device_name: mps
2025-02-07 14:57:02,222 - INFO - Load pretrained SentenceTransformer: Alibaba-NLP/gte-modernbert-base
2025-02-07 14:57:07,139 - INFO - Load pretrained sentiment analysis model: finiteautomata/bertweet-base-sentiment-analysis.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Device set to use mps:0
2025-02-07 14:57:08,362 - INFO - Starting graph construction for dataset MOLWENI...
Processing dialogs:   0%|          | 0/500 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing dialogs:   0%|          | 0/500 [00:01<?, ?it/s]


KeyError: 'NEG'

In [ ]:
builder.display_graph(0)